In [1]:
# Import Splinter, BeautifulSoup and dependencies
import requests
from bs4 import BeautifulSoup as bs
from pymongo import MongoClient
from splinter import Browser
import time

In [3]:
departure = MCI
arrival = []
from_date = []
to_date = []

In [2]:
browser = Browser('chrome')
browser.visit('https://www.kayak.com/flights/MCI-SAN/2024-07-04/2024-07-11')
time.sleep(1)

In [3]:
# Create a Beautiful Soup object
soup = bs(browser.html,'html.parser')

In [67]:
# Extract all flight text elements
for i in soup.find_all(id="listWrapper")[0].find_all('div', class_ = "nrc6 nrc6-mod-pres-multi-fare"):
    print(i.text)

Let’s go. We’ve got places to see.Plan your next trip today.5:45 am – 9:01 amDelta1 stopSLC5h 16mMCI-SAN6:15 am – 1:53 pmDelta1 stopLAX5h 38mSAN-MCIOperated by Skywest DBA Delta ConnectionAd$305Basic EconomyDeltaView Deal$385Main CabinDeltaView Deal
Best5:13 pm – 8:40 pmUnited Airlines1 stopDEN5h 27mMCI-SAN4:10 pm – 12:29 am+1United Airlines1 stopSFO6h 19mSAN-MCI$218Basic EconomyUnited AirlinesView Deal$298EconomyUnited AirlinesView Deal
Cheapest7:44 pm – 7:07 am+1Frontier1 stopDEN13h 23mMCI-SAN10:20 pm – 6:12 am+1Spirit Airlines1 stopLAS5h 52mSAN-MCI$197EconomyBooking.comView Deal
5:59 am – 8:52 amAmerican Airlines1 stopPHX4h 53mMCI-SAN3:31 pm – 11:20 pmAmerican Airlines1 stopDFW5h 49mSAN-MCI$218Basic EconomyAmerican AirlinesView Deal$298Main CabinAmerican AirlinesView Deal
5:59 am – 8:52 amAmerican Airlines1 stopPHX4h 53mMCI-SAN4:20 pm – 11:50 pmAmerican Airlines1 stopPHX5h 30mSAN-MCI$244Basic EconomyAmerican AirlinesView Deal$324Main CabinAmerican AirlinesView Deal
2:22 pm – 6:08 pm

In [79]:
# Extract flight times
flight_times_list = []

for flight in soup.find_all('div', class_='VY2U'):  # Adjust class as needed
    flight_times = flight.find_all('span')  # Find all span elements within the flight element
    departure_time = flight_times[0].text.strip()  # Extract departure time
    arrival_time = flight_times[2].text.strip()  # Extract arrival time
    airline_name = flight.find('div', class_='c_cgF c_cgF-mod-variant-default')
    airline = airline_name.text.strip()
    flight_times_list.append({
        "departure_time": departure_time,
        "arrival_time": arrival_time,
        "airline": airline
    })

print(flight_times_list)

[{'departure_time': '5:45 am', 'arrival_time': '9:01 am', 'airline': 'Delta'}, {'departure_time': '6:15 am', 'arrival_time': '1:53 pm', 'airline': 'Delta'}, {'departure_time': '5:13 pm', 'arrival_time': '8:40 pm', 'airline': 'United Airlines'}, {'departure_time': '4:10 pm', 'arrival_time': '12:29 am+1', 'airline': 'United Airlines'}, {'departure_time': '7:44 pm', 'arrival_time': '7:07 am+1', 'airline': 'Frontier'}, {'departure_time': '10:20 pm', 'arrival_time': '6:12 am+1', 'airline': 'Spirit Airlines'}, {'departure_time': '5:59 am', 'arrival_time': '8:52 am', 'airline': 'American Airlines'}, {'departure_time': '3:31 pm', 'arrival_time': '11:20 pm', 'airline': 'American Airlines'}, {'departure_time': '5:59 am', 'arrival_time': '8:52 am', 'airline': 'American Airlines'}, {'departure_time': '4:20 pm', 'arrival_time': '11:50 pm', 'airline': 'American Airlines'}, {'departure_time': '2:22 pm', 'arrival_time': '6:08 pm', 'airline': 'American Airlines'}, {'departure_time': '3:31 pm', 'arrival

In [80]:
# Extract flight durations
flight_durations_list = []

for i in soup.find_all(id="listWrapper")[0].find_all('div', class_="nrc6 nrc6-mod-pres-multi-fare"):
    duration_route = i.find_all('div', class_='xdW8')
    outbound_flight_duration = duration_route[0].find('div', class_='vmXl vmXl-mod-variant-default').text.strip()
    return_flight_duration = duration_route[1].find('div', class_='vmXl vmXl-mod-variant-default').text.strip()
    flight_durations_list.append({
        "outbound_flight_duration": outbound_flight_duration,
        "return_flight_duration": return_flight_duration
    })

print(flight_durations_list)

[{'outbound_flight_duration': '5h 16m', 'return_flight_duration': '5h 38m'}, {'outbound_flight_duration': '5h 27m', 'return_flight_duration': '6h 19m'}, {'outbound_flight_duration': '13h 23m', 'return_flight_duration': '5h 52m'}, {'outbound_flight_duration': '4h 53m', 'return_flight_duration': '5h 49m'}, {'outbound_flight_duration': '4h 53m', 'return_flight_duration': '5h 30m'}, {'outbound_flight_duration': '5h 46m', 'return_flight_duration': '5h 49m'}, {'outbound_flight_duration': '5h 46m', 'return_flight_duration': '5h 30m'}, {'outbound_flight_duration': '5h 52m', 'return_flight_duration': '6h 19m'}, {'outbound_flight_duration': '5h 17m', 'return_flight_duration': '6h 19m'}, {'outbound_flight_duration': '5h 27m', 'return_flight_duration': '5h 40m'}, {'outbound_flight_duration': '6h 08m', 'return_flight_duration': '6h 19m'}, {'outbound_flight_duration': '5h 52m', 'return_flight_duration': '5h 40m'}, {'outbound_flight_duration': '5h 17m', 'return_flight_duration': '5h 40m'}, {'outbound

In [81]:
# Extract flight routes
flight_routes_list = []

for i in soup.find_all(id="listWrapper")[0].find_all('div', class_="nrc6 nrc6-mod-pres-multi-fare"):
    duration_route = i.find_all('div', class_='xdW8')
    outbound_route = duration_route[0].find('div', class_='EFvI').text.strip()
    return_route = duration_route[1].find('div', class_='EFvI').text.strip()
    flight_routes_list.append({
        "outbound_route": outbound_route,
        "return_route": return_route
    })
print(flight_routes_list)

[{'outbound_route': 'MCI-SAN', 'return_route': 'SAN-MCI'}, {'outbound_route': 'MCI-SAN', 'return_route': 'SAN-MCI'}, {'outbound_route': 'MCI-SAN', 'return_route': 'SAN-MCI'}, {'outbound_route': 'MCI-SAN', 'return_route': 'SAN-MCI'}, {'outbound_route': 'MCI-SAN', 'return_route': 'SAN-MCI'}, {'outbound_route': 'MCI-SAN', 'return_route': 'SAN-MCI'}, {'outbound_route': 'MCI-SAN', 'return_route': 'SAN-MCI'}, {'outbound_route': 'MCI-SAN', 'return_route': 'SAN-MCI'}, {'outbound_route': 'MCI-SAN', 'return_route': 'SAN-MCI'}, {'outbound_route': 'MCI-SAN', 'return_route': 'SAN-MCI'}, {'outbound_route': 'MCI-SAN', 'return_route': 'SAN-MCI'}, {'outbound_route': 'MCI-SAN', 'return_route': 'SAN-MCI'}, {'outbound_route': 'MCI-SAN', 'return_route': 'SAN-MCI'}, {'outbound_route': 'MCI-SAN', 'return_route': 'SAN-MCI'}, {'outbound_route': 'MCI-SAN', 'return_route': 'SAN-MCI'}, {'outbound_route': 'MCI-SAN', 'return_route': 'SAN-MCI'}, {'outbound_route': 'MCI-SAN', 'return_route': 'SAN-MCI'}]


In [88]:
# Extract price information
price_list = []

for i in soup.find_all(id="listWrapper")[0].find_all('div', class_="nrc6 nrc6-mod-pres-multi-fare"):
    # Extract all price text elements
    price_texts = i.find_all('div', class_="f8F1-price-text")
    
    for price_text in price_texts:
        price = price_text.text.strip()
        price_list.append(price)

print(price_list)

['$305', '$385', '$218', '$298', '$197', '$218', '$298', '$244', '$324', '$244', '$324', '$270', '$350', '$218', '$298', '$258', '$338', '$299', '$379', '$218', '$298', '$299', '$379', '$338', '$418', '$330', '$410', '$265', '$345', '$299', '$379', '$244', '$324']


In [89]:
# Extract price information
tier_list = []

for i in soup.find_all(id="listWrapper")[0].find_all('div', class_="nrc6 nrc6-mod-pres-multi-fare"):
    # Extract all price text elements
    tier_texts = i.find_all('div', class_="aC3z-name aC3z-mod-ellipsis")
    
    for tier_text in tier_texts:
        tier = tier_text.text.strip()
        tier_list.append(tier)

print(tier_list)

['Basic Economy', 'Main Cabin', 'Basic Economy', 'Economy', 'Economy', 'Basic Economy', 'Main Cabin', 'Basic Economy', 'Main Cabin', 'Basic Economy', 'Main Cabin', 'Basic Economy', 'Main Cabin', 'Basic Economy', 'Economy', 'Basic Economy', 'Economy', 'Basic Economy', 'Economy', 'Basic Economy', 'Economy', 'Basic Economy', 'Economy', 'Basic Economy', 'Economy', 'Basic Economy', 'Economy', 'Basic Economy', 'Economy', 'Basic Economy', 'Economy', 'Basic Economy', 'Main Cabin']


In [90]:
# Extract all flight text elements
for i in soup.find_all(id="listWrapper")[0].find_all('div', class_="nrc6 nrc6-mod-pres-multi-fare"):
    flights = i.find_all('div', class_='VY2U')  # Find all flight elements within the class
    for flight in flights:
        flight_times = flight.find_all('span')  # Find all span elements within the flight element
        if len(flight_times) >= 3:  # Ensure there are at least three span elements
            departure_time = flight_times[0].text.strip()  # Extract departure time
            arrival_time = flight_times[2].text.strip()  # Extract arrival time
            print("Departure Time:", departure_time)
            print("Arrival Time:", arrival_time)
        else:
            print("Unable to extract flight times")
    duration_route = i.find_all('div', class_='xdW8')
    for dr in duration_route:
        # Extract flight duration
        flight_duration = dr.find('div', class_='vmXl vmXl-mod-variant-default').text.strip()
        print(flight_duration)

        # Extract flight route
        flight_route = dr.find('div', class_='EFvI').text.strip()
        print(flight_route)


Departure Time: 5:45 am
Arrival Time: 9:01 am
Departure Time: 6:15 am
Arrival Time: 1:53 pm
5h 16m
MCI-SAN
5h 38m
SAN-MCI
Departure Time: 5:13 pm
Arrival Time: 8:40 pm
Departure Time: 4:10 pm
Arrival Time: 12:29 am+1
5h 27m
MCI-SAN
6h 19m
SAN-MCI
Departure Time: 7:44 pm
Arrival Time: 7:07 am+1
Departure Time: 10:20 pm
Arrival Time: 6:12 am+1
13h 23m
MCI-SAN
5h 52m
SAN-MCI
Departure Time: 5:59 am
Arrival Time: 8:52 am
Departure Time: 3:31 pm
Arrival Time: 11:20 pm
4h 53m
MCI-SAN
5h 49m
SAN-MCI
Departure Time: 5:59 am
Arrival Time: 8:52 am
Departure Time: 4:20 pm
Arrival Time: 11:50 pm
4h 53m
MCI-SAN
5h 30m
SAN-MCI
Departure Time: 2:22 pm
Arrival Time: 6:08 pm
Departure Time: 3:31 pm
Arrival Time: 11:20 pm
5h 46m
MCI-SAN
5h 49m
SAN-MCI
Departure Time: 2:22 pm
Arrival Time: 6:08 pm
Departure Time: 4:20 pm
Arrival Time: 11:50 pm
5h 46m
MCI-SAN
5h 30m
SAN-MCI
Departure Time: 1:45 pm
Arrival Time: 5:37 pm
Departure Time: 4:10 pm
Arrival Time: 12:29 am+1
5h 52m
MCI-SAN
6h 19m
SAN-MCI
Departur

In [9]:
# Extract all flight text elements
for i in soup.find_all(id="listWrapper")[0].find_all('div', class_ = "hJSA"): 
    #print(i.next_element.next_element.next_element.next_element.find('span'))
    #print(i.find('hJSA-item'))
    print(i.text)
    #<div data-resultid=, class="nrc6"

5:45 am – 9:01 amDelta1 stopSLC5h 16mMCI-SAN6:15 am – 1:53 pmDelta1 stopLAX5h 38mSAN-MCI
5:13 pm – 8:40 pmUnited Airlines1 stopDEN5h 27mMCI-SAN4:10 pm – 12:29 am+1United Airlines1 stopSFO6h 19mSAN-MCI
7:44 pm – 7:07 am+1Frontier1 stopDEN13h 23mMCI-SAN10:20 pm – 6:12 am+1Spirit Airlines1 stopLAS5h 52mSAN-MCI
5:59 am – 8:52 amAmerican Airlines1 stopPHX4h 53mMCI-SAN3:31 pm – 11:20 pmAmerican Airlines1 stopDFW5h 49mSAN-MCI
5:59 am – 8:52 amAmerican Airlines1 stopPHX4h 53mMCI-SAN4:20 pm – 11:50 pmAmerican Airlines1 stopPHX5h 30mSAN-MCI
2:22 pm – 6:08 pmAmerican Airlines1 stopPHX5h 46mMCI-SAN3:31 pm – 11:20 pmAmerican Airlines1 stopDFW5h 49mSAN-MCI
2:22 pm – 6:08 pmAmerican Airlines1 stopPHX5h 46mMCI-SAN4:20 pm – 11:50 pmAmerican Airlines1 stopPHX5h 30mSAN-MCI
1:45 pm – 5:37 pmUnited Airlines1 stopDEN5h 52mMCI-SAN4:10 pm – 12:29 am+1United Airlines1 stopSFO6h 19mSAN-MCI
6:15 am – 9:32 amUnited Airlines1 stopDEN5h 17mMCI-SAN4:10 pm – 12:29 am+1United Airlines1 stopSFO6h 19mSAN-MCI
5:13 pm – 8

In [10]:
# Extract all flight text elements
for i in soup.find_all(id="listWrapper")[0].find_all('div', class_ = "M_JD-large-display"):
    #print(i.next_element.next_element.next_element.next_element.find('span'))
    #print(i.find('hJSA-item'))
    print(i.text)


$305Basic EconomyDelta
$385Main CabinDelta
$218Basic EconomyUnited Airlines
$298EconomyUnited Airlines
$197EconomyBooking.com
$218Basic EconomyAmerican Airlines
$298Main CabinAmerican Airlines
$244Basic EconomyAmerican Airlines
$324Main CabinAmerican Airlines
$244Basic EconomyAmerican Airlines
$324Main CabinAmerican Airlines
$270Basic EconomyAmerican Airlines
$350Main CabinAmerican Airlines
$218Basic EconomyUnited Airlines
$298EconomyUnited Airlines
$258Basic EconomyUnited Airlines
$338EconomyUnited Airlines
$299Basic EconomyUnited Airlines
$379EconomyUnited Airlines
$218Basic EconomyUnited Airlines
$298EconomyUnited Airlines
$299Basic EconomyUnited Airlines
$379EconomyUnited Airlines
$338Basic EconomyUnited Airlines
$418EconomyUnited Airlines
$330Basic EconomyUnited Airlines
$410EconomyUnited Airlines
$265Basic EconomyUnited Airlines
$345EconomyUnited Airlines
$299Basic EconomyUnited Airlines
$379EconomyUnited Airlines
$244Basic EconomyAmerican Airlines
$324Main CabinAmerican Airlines

In [40]:
i.__dict__

{'parser_class': bs4.BeautifulSoup,
 'name': 'div',
 'namespace': None,
 '_namespaces': {},
 'prefix': None,
 'sourceline': 153,
 'sourcepos': 597445,
 'known_xml': False,
 'attrs': {'class': ['OrUx-disclaimer-text']},
 'contents': ["¹Hacker Fare tickets are sold to/from a destination via different airlines and are subject to each airline's booking requirements and terms, including changes to itineraries. Any changes made to one of your tickets will not necessarily afford rights to change the other ticket. Fares change frequently and are subject to availability. International travel may require proof of return flight."],
 'parent': <div class="OrUx"><div class="OrUx-disclaimer-text"><div class="OrUx-asterisk">*</div><div class="OrUx-false"><span class="OrUx-daybase">Prices are per person and do not include baggage fees. Some train and bus prices may not include a service fee added by the provider at checkout.</span><span class="OrUx-total">Prices show the total for all passengers and d

In [15]:
browser.html[79067-10:]

'<a href="#listWrapper">Go to flights results section</a></nav><div class="kml-layout ev1_-content ev1_-mod-layout-multi-fare ev1_-mod-rr-default-disabled"><div class="ev1_-list-view"><div class="ev1_-left-rail" role="form" tabindex="-1" id="leftRail"><div class="e_0j"><div><div class="VuYm"><div class="r9rr"><div class="r9rr-icon-wrapper"><svg xmlns="http://www.w3.org/2000/svg" width="20" height="20" viewBox="0 0 200 200" role="presentation"><path fill="#000" d="M 115.231 75.457 C 91.876 71.115 73.612 52.702 69.271 29.498 C 69.12 28.299 67.46 28.299 67.324 29.498 C 62.982 52.851 44.569 71.115 21.366 75.352 C 20.167 75.608 20.167 77.253 21.366 77.404 C 44.718 81.745 62.982 100.159 67.324 123.363 C 67.475 124.56 69.12 124.56 69.271 123.363 C 73.612 100.009 92.026 81.745 115.231 77.404 C 116.428 77.253 116.428 75.756 115.231 75.457 Z"></path><path fill="#000" d="M 131.81 148.07 C 120.11 145.87 110.91 136.67 108.71 124.97 C 108.61 124.37 107.81 124.37 107.71 124.97 C 105.51 136.67 96.31 1

In [ ]:
#listWrapper > div > div:nth-child(2) > div > div:nth-child(2)

In [51]:
browser.quit()